<a href="https://colab.research.google.com/github/andreaderuvo/rc_car_tracking/blob/main/C8_train_deepsort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install tensorflow-gpu
%tensorflow_version 1.x
!pip install tensorflow-gpu==1.15.0

In [ ]:
#import tensorflow and check version
import tensorflow
print(tensorflow.__version__)

In [ ]:
#configuration
PROJECT = 'rc_car_tracking'
MODEL = 'deepsort'

COLAB_BASE_PROJECT = f'/content/{PROJECT}'
DRIVE_MOUNT_FOLDER = 'drive'
DRIVE_BASE_PROJECT = f'{DRIVE_MOUNT_FOLDER}/MyDrive/{PROJECT}'
DRIVE_TRAIN_FOLDER = f'{MODEL}_train'
DRIVE_DATASET_FOLDER = f'{MODEL}_dataset'

TRAIN_BATCH_SIZE=8
TRAIN_NUMBER_OF_STEPS=20000

In [ ]:
import os
import shutil

In [ ]:
#mount Google Drive
from google.colab import drive
drive.mount(DRIVE_MOUNT_FOLDER, force_remount=True)

In [ ]:
#symbolic link pointing the google drive base project
!ln -s $DRIVE_BASE_PROJECT $COLAB_BASE_PROJECT

In [ ]:
os.makedirs(os.path.join(COLAB_BASE_PROJECT, DRIVE_TRAIN_FOLDER), exist_ok = True)

In [ ]:
#cloning the cosine_metric_learning repo
!git clone https://github.com/nwojke/cosine_metric_learning

In [ ]:
%cd cosine_metric_learning/

In [ ]:
#download and place customized files for custom dataset
!wget https://raw.githubusercontent.com/andreaderuvo/rc_car_tracking/main/cosine_metric_learning/datasets/rc.py
!mv rc.py datasets/rc.py
!wget https://raw.githubusercontent.com/andreaderuvo/rc_car_tracking/main/cosine_metric_learning/train_rc.py
!wget https://raw.githubusercontent.com/andreaderuvo/rc_car_tracking/main/cosine_metric_learning/train_app.py

In [ ]:
!rm -rf $COLAB_BASE_PROJECT/$DRIVE_TRAIN_FOLDER/log/train

In [ ]:
#train
!python train_rc.py --dataset_dir=$COLAB_BASE_PROJECT/$DRIVE_DATASET_FOLDER --loss_mode=cosine-softmax --log_dir=$COLAB_BASE_PROJECT/$DRIVE_TRAIN_FOLDER/log/train --run_id=cosine-softmax --batch_size=$TRAIN_BATCH_SIZE --number_of_steps=$TRAIN_NUMBER_OF_STEPS

In [ ]:
%load_ext tensorboard

%tensorboard --logdir $COLAB_BASE_PROJECT/$DRIVE_TRAIN_FOLDER/log/train

In [ ]:
#!python train_veri.py --mode='finalize' --restore_path='/content/log/rc/cosine-softmax/model.ckpt-0' --batch_size=2

In [ ]:
!mkdir $COLAB_BASE_PROJECT/$DRIVE_TRAIN_FOLDER/frozen
!python train_rc.py --restore_path=$COLAB_BASE_PROJECT/$DRIVE_TRAIN_FOLDER/log/train/cosine-softmax/model.ckpt-10000 --mode=freeze --frozen_output=$COLAB_BASE_PROJECT/$DRIVE_TRAIN_FOLDER/frozen/rccar.pb